In [488]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [489]:
train= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
print("shape of train :",train.shape)
print("shape of test :",test.shape)

shape of train : (1460, 81)
shape of test : (1459, 80)


In [490]:
X_train=train.drop(columns="SalePrice",axis=1)
y_train=train["SalePrice"]

print("SHAPE OF X_TRAIN :" , X_train.shape)
print("SHAPE OF y_train :" , y_train.shape)

SHAPE OF X_TRAIN : (1460, 80)
SHAPE OF y_train : (1460,)


# Missing Value  Imputation

- finding missing value columns


In [491]:
isnull_sum=X_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

- separating numerical column having null values

In [492]:
num_vars=X_train.select_dtypes(include=['int64','float64']).columns
num_var_miss=[]
for var in num_vars:
    if isnull_sum[var]>0:
        num_var_miss.append(var)
num_var_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

- separating categorical variable having null values

In [493]:
cat_vars=X_train.select_dtypes(include=['object']).columns
cat_var_miss=[]
for var in cat_vars:
    if isnull_sum[var]>0:
        cat_var_miss.append(var)
cat_var_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

- creating pipeline

In [494]:
num_var_mean=['LotFrontage']
num_var_median=['MasVnrArea', 'GarageYrBlt']
cat_var_mode=['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu']
cat_var_missing=['GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

- creating strategy

In [495]:
num_var_mean_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="mean"))])
num_var_median_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median"))])
cat_var_mode_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])
cat_var_constant_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="constant" ,fill_value="missing"))])


- now we will tell on what variable(columns) we apply above startegies

In [496]:
preprocessor=ColumnTransformer(transformers=[("mean_imputer",num_var_mean_imputer,num_var_mean),
                               ("median_imputer",num_var_median_imputer,num_var_median),
                               ("mode_imputer",cat_var_mode_imputer,cat_var_mode),
                               ("constant_imputer",cat_var_constant_imputer,cat_var_missing)
                              ]
                 )

In [497]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_imputer',
                                 Pip

In [498]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_i

- now if we want to see what values are imputed in columns

In [499]:
#which value is imputed in "num_var_mean" columns
preprocessor.named_transformers_["mean_imputer"].named_steps["imputer"].statistics_

array([70.04995837])

In [500]:
#which value is imputed in "num_var_median" columns
preprocessor.named_transformers_["median_imputer"].named_steps["imputer"].statistics_

array([   0., 1980.])

In [501]:
#which value is imputed in "cat_var_mode" columns
preprocessor.named_transformers_["mode_imputer"].named_steps["imputer"].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

#### now to impute all above values in X_train dataset
#### we use transform method of imputer class
- one disadvantage of this methodwe give it dataframe and it returns 2d-array

- but we convert that dataset again in Dataframe by just one line of code

In [502]:
X_test = test.copy()
X_train_clean=preprocessor.transform(X_train)
X_test_clean=preprocessor.transform(X_test)


In [503]:
X_test_clean

array([[80.0, 0.0, 1961.0, ..., 'missing', 'MnPrv', 'missing'],
       [81.0, 108.0, 1958.0, ..., 'missing', 'missing', 'Gar2'],
       [74.0, 0.0, 1997.0, ..., 'missing', 'MnPrv', 'missing'],
       ...,
       [160.0, 0.0, 1960.0, ..., 'missing', 'missing', 'missing'],
       [62.0, 0.0, 1980.0, ..., 'missing', 'MnPrv', 'Shed'],
       [74.0, 94.0, 1993.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [504]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('constant_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
  

In [505]:
X_train_clean_miss_var =pd.DataFrame(X_train_clean,columns=num_var_mean+num_var_median+cat_var_mode+cat_var_missing)

In [506]:
isnull_sumtest=X_test.isnull().sum()
isnull_sumtest

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [507]:
remainder_vars_index = [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   61,
   62,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   75,
   76,
   77,
   78,
   79]

In [508]:
remainder_vars = []
for var_index in remainder_vars_index:
    remainder_vars.append(isnull_sumtest.keys()[var_index])

In [509]:
X_train =  pd.concat([X_train_clean_miss_var,train[remainder_vars]], axis=1)

In [510]:
X_train.isnull().sum()

LotFrontage      0
MasVnrArea       0
GarageYrBlt      0
Alley            0
MasVnrType       0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64

In [511]:
X_train.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,65,196,2003,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,...,61,0,0,0,0,0,2,2008,WD,Normal
1,80,0,1976,Grvl,None,Gd,TA,Gd,ALQ,Unf,...,0,0,0,0,0,0,5,2007,WD,Normal
2,68,162,2001,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,...,42,0,0,0,0,0,9,2008,WD,Normal
3,60,0,1998,Grvl,None,TA,Gd,No,ALQ,Unf,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,84,350,2000,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,...,84,0,0,0,0,0,12,2008,WD,Normal


In [512]:
# Create test DataFrame with missing value imputed variables
X_test_clean_miss_var = pd.DataFrame(X_test_clean, columns=num_var_mean+num_var_median+cat_var_mode+cat_var_missing)
X_test_clean_miss_var.shape

(1459, 19)

In [513]:
# concatinate X_test_clean_miss_var df and remainder_vars
X_test =  pd.concat([X_test_clean_miss_var,test[remainder_vars]], axis=1)
X_test.shape

(1459, 80)

In [514]:
isnull_sum_test = X_test.isnull().sum()
isnull_sum_test

LotFrontage      0
MasVnrArea       0
GarageYrBlt      0
Alley            0
MasVnrType       0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64

In [515]:
X_test.isnull().sum().sum()

22

In [516]:
# 22 <= What is this, in X_test df still missing values as available but why 
#because we fill missing values in those columns which have missing value present in only X_train df
# Basicaly we get df then  find missing values variables then split df into X_train, X_test, y_train, y_test
# after that we fill missing value

# so if you have train and test df seperatly then first thing you should concatinate then find the missing 
# values variables it's is great strategy and carry on
# so you can try yourself

In [517]:
isnull_sumtest=X_test.isnull().sum()
isnull_sumtest

LotFrontage      0
MasVnrArea       0
GarageYrBlt      0
Alley            0
MasVnrType       0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64

In [518]:
# finding the numerical variable which have mising value
num_vars_test = X_test.select_dtypes(include=["int64", "float64"]).columns
num_vars_miss_test = [var for var in num_vars_test if isnull_sum_test[var]>0]
num_vars_miss_test

['BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageCars',
 'GarageArea']

In [519]:
# finding the categorical variable which have mising value
cat_vars_test = X_test.select_dtypes(include=["O"]).columns
cat_vars_miss_test = [var for var in cat_vars_test if isnull_sum_test[var]>0]
cat_vars_miss_test

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'KitchenQual',
 'Functional',
 'SaleType']

In [520]:
num_var_mean_imputer_test=Pipeline(steps=[("imputer",SimpleImputer(strategy="mean"))])

cat_var_mode_imputer_test=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])



In [521]:
preprocessor_test=ColumnTransformer(transformers=[("mean_imputer",num_var_mean_imputer_test,num_vars_miss_test),
                            
                               ("mode_imputer",cat_var_mode_imputer_test,cat_vars_miss_test),
                              
                              ]
                 )

In [522]:
preprocessor_test.fit(X_test)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                                  'GarageCars', 'GarageArea']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['MSZoning', 'Utilities', 'Exterior1st',
                                  'Exterior2nd', 'KitchenQual', 'Functional',
                                  'SaleType'])])

In [523]:
preprocessor_test.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                                  'GarageCars', 'GarageArea']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['MSZoning', 'Utilities', 'Exterior1st',
                                  'Exterior2nd', 'KitchenQual', 'Functional',
                                  'SaleType'])])>

In [524]:
#which value is imputed in "cat_var_mode" columns
preprocessor_test.named_transformers_["mode_imputer"].named_steps["imputer"].statistics_

array(['RL', 'AllPub', 'VinylSd', 'VinylSd', 'TA', 'Typ', 'WD'],
      dtype=object)

In [525]:
X_test_clean_test2=preprocessor_test.transform(X_test)

In [526]:
X_test_clean_test2

array([[468.0, 144.0, 270.0, ..., 'TA', 'Typ', 'WD'],
       [923.0, 0.0, 406.0, ..., 'Gd', 'Typ', 'WD'],
       [791.0, 0.0, 137.0, ..., 'TA', 'Typ', 'WD'],
       ...,
       [1224.0, 0.0, 0.0, ..., 'TA', 'Typ', 'WD'],
       [337.0, 0.0, 575.0, ..., 'TA', 'Typ', 'WD'],
       [758.0, 0.0, 238.0, ..., 'TA', 'Typ', 'WD']], dtype=object)

In [527]:
preprocessor_test.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['BsmtFinSF1',
   'BsmtFinSF2',
   'BsmtUnfSF',
   'TotalBsmtSF',
   'BsmtFullBath',
   'BsmtHalfBath',
   'GarageCars',
   'GarageArea']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['MSZoning',
   'Utilities',
   'Exterior1st',
   'Exterior2nd',
   'KitchenQual',
   'Functional',
   'SaleType']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   22,
   23,
   24,
   25,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   42,
   43,
   44,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   58,
   59,
   60,
   61,
   63,
   65,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   79])]

In [528]:
X_test_clean_miss_var_test2 =pd.DataFrame(X_test_clean_test2,columns=num_vars_miss_test+cat_vars_miss_test)
X_test_clean_miss_var_test2.isnull().sum().sum()

0

In [529]:
X_test_clean_miss_var_test2

,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageCars,GarageArea,MSZoning,Utilities,Exterior1st,Exterior2nd,KitchenQual,Functional,SaleType
0,468,144,270,882,0,0,1,730,RH,AllPub,VinylSd,VinylSd,TA,Typ,WD
1,923,0,406,1329,0,0,1,312,RL,AllPub,Wd Sdng,Wd Sdng,Gd,Typ,WD
2,791,0,137,928,0,0,2,482,RL,AllPub,VinylSd,VinylSd,TA,Typ,WD
3,602,0,324,926,0,0,2,470,RL,AllPub,VinylSd,VinylSd,Gd,Typ,WD
4,263,0,1017,1280,0,0,2,506,RL,AllPub,HdBoard,HdBoard,Gd,Typ,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0,0,546,546,0,0,0,0,RM,AllPub,CemntBd,CmentBd,TA,Typ,WD
1455,252,0,294,546,0,0,1,286,RM,AllPub,CemntBd,CmentBd,TA,Typ,WD
1456,1224,0,0,1224,1,0,2,576,RL,AllPub,VinylSd,VinylSd,TA,Typ,WD
1457,337,0,575,912,0,1,0,0,RL,AllPub,HdBoard,Wd Shng,TA,Typ,WD


In [530]:
remainder_vars_test =  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   22,
   23,
   24,
   25,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   42,
   43,
   44,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   58,
   59,
   60,
   61,
   63,
   65,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   79]
remainder=[]
for var_index in remainder_vars_test:
    remainder.append(isnull_sumtest.keys()[var_index])
    print(isnull_sumtest.keys()[var_index])

LotFrontage
MasVnrArea
GarageYrBlt
Alley
MasVnrType
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Electrical
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature
Id
MSSubClass
LotArea
Street
LotShape
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
ExterQual
ExterCond
Foundation
Heating
HeatingQC
CentralAir
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold
SaleCondition


In [531]:
X_test[remainder].isnull().sum().sum()

0

In [532]:
#X_test =  pd.concat([X_test_clean_miss_var_test2,test[remainder]], axis=1)

In [533]:
X_test =  pd.concat([X_test_clean_miss_var_test2,X_test[remainder]], axis=1)

In [534]:
X_test.isnull().sum().sum()

0